# Deep Learning classification using the fast.ai library built on top of PyTorch

In [ ]:
from fastai.text import *
from fastai.metrics import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
path = "/root/"
torch.cuda.empty_cache()
!/opt/bin/nvidia-smi

In [ ]:
# Import data. There are two clases of data, one for determining the language model (how to read the words), and the other for classification (into ratings 1-10)

data_lm = TextLMDataBunch.from_csv(path, "train.tsv", delimiter="\t", text_cols="review", label_cols="rating", bs=96)
data_clas = TextClasDataBunch.from_csv(path, "train.tsv", delimiter="\t", text_cols="review", label_cols="rating", vocab=data_lm.train_ds.vocab, bs=96)   

In [ ]:
# Save databunch
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [ ]:
# Load Databunch

bs = 48
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

In [ ]:
# Setup language model learner using the AWD LSTM model.

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=.05)

In [ ]:
#fit the model to the data using the one cycle policy

learn.fit_one_cycle(1, 1e-3)

In [ ]:
# find the losses at different learning rates and plot the curve

learn.lr_find();
learn.recorder.plot()

In [ ]:
# example prediction using the fitted language model

learn.predict("it could have been", n_words=10)

In [ ]:
# save the trained weights of the model

learn.save("fit_head")
learn.load("fit_head");

In [ ]:
# unfreeze the entire model and train again
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-2, 1e-1))

In [ ]:
# save the encoder from the fitted language model
learn.save_encoder("ft_enc")

In [ ]:
# Setup the classification learning model and load the language model encoder

learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics = [accuracy, rmse])
learn.load_encoder("ft_enc");

In [ ]:
# Preview the classification batch. It contains one column for the tokenized reviews and another column for the rating labels

data_clas.show_batch()

In [ ]:
# Train model using the fit one cycle policy

learn.fit_one_cycle(2)

In [ ]:
# save these initally trained weight

learn.save("first_clas")
learn.load("first_clas");

In [ ]:
# freeeze the model excep the last two layers for retraining

learn.freeze_to(-2)

In [ ]:
# Plot the learning rate 

learn.lr_find()
learn.recorder.plot()

In [ ]:
# Fit partially unfrozen model

learn.fit_one_cycle(1, slice(1e-4, 1e-3))

In [ ]:
# fit fully unfrozen model

learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-3))

In [ ]:
# Save the final classification weights.

learn.save("final_clas")
learn.load("final_clas");